# <center>PREDICTING EARNING MANIPULATIONS BY INDIAN FIRMS</center>

<span style='font-family:"Times New Roman"'><font size = 4>Installing necessary packages & libraries</font></span>

In [1]:
install.packages(c("xlsx" , "caret", "e1071" , "ROSE" , "party" , "rpart.plot", "randomForest", "ROCR"))
library(caret)
library(e1071)
library(ROSE)
library(xlsx)
library(rpart)
library(rpart.plot)
library(randomForest)
library(ROCR)

also installing the dependencies ‘pillar’, ‘glue’, ‘tibble’, ‘ellipsis’, ‘dplyr’, ‘tidyselect’, ‘vctrs’, ‘TH.data’, ‘lifecycle’, ‘rlang’, ‘tidyr’, ‘libcoin’, ‘matrixStats’, ‘multcomp’, ‘gdata’, ‘ModelMetrics’, ‘recipes’, ‘pROC’, ‘modeltools’, ‘strucchange’, ‘coin’, ‘gplots’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Loading required package: lattice
Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Loaded ROSE 0.0-3

randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:ggplot2’:

    margin



In [9]:
library(projectLib)
project <- projectLib::Project$new(projectId="192a265a-e9ec-418a-aa6e-36e58abd2250", projectToken="p-18202afc4297e6340c012a64b817b39a6d803f5d")
my.file <- project$get_file("Manipulations.csv")

In [10]:
# Importing the dataset
ML <- read.csv(text = rawToChar(my.file))

In [11]:
options(scipen = 99)
sum(complete.cases(ML))/nrow(ML)*100
#No missing cases

[1] 100

In [12]:
str(ML)

'data.frame':	1239 obs. of  11 variables:
 $ Company.ID   : int  1 2 3 4 5 6 7 8 9 10 ...
 $ DSRI         : num  1.62 1 1 1.49 1 ...
 $ GMI          : num  1.13 1.61 1.02 1 1.37 ...
 $ AQI          : num  7.185 1.005 1.241 0.466 0.637 ...
 $ SGI          : num  0.366 13.081 1.475 0.673 0.861 ...
 $ DEPI         : num  1.38 0.4 1.17 2 1.45 ...
 $ SGAI         : num  1.6241 5.1982 0.6477 0.0929 1.7415 ...
 $ ACCR         : num  -0.1668 0.0605 0.0367 0.2734 0.123 ...
 $ LEVI         : num  1.161 0.987 1.264 0.681 0.939 ...
 $ Manipulater  : Factor w/ 2 levels "No","Yes": 2 2 2 2 2 2 2 2 2 2 ...
 $ C.MANIPULATOR: int  1 1 1 1 1 1 1 1 1 1 ...


In [13]:
ML$C.MANIPULATOR <- as.factor(ML$C.MANIPULATOR)
str(ML)

'data.frame':	1239 obs. of  11 variables:
 $ Company.ID   : int  1 2 3 4 5 6 7 8 9 10 ...
 $ DSRI         : num  1.62 1 1 1.49 1 ...
 $ GMI          : num  1.13 1.61 1.02 1 1.37 ...
 $ AQI          : num  7.185 1.005 1.241 0.466 0.637 ...
 $ SGI          : num  0.366 13.081 1.475 0.673 0.861 ...
 $ DEPI         : num  1.38 0.4 1.17 2 1.45 ...
 $ SGAI         : num  1.6241 5.1982 0.6477 0.0929 1.7415 ...
 $ ACCR         : num  -0.1668 0.0605 0.0367 0.2734 0.123 ...
 $ LEVI         : num  1.161 0.987 1.264 0.681 0.939 ...
 $ Manipulater  : Factor w/ 2 levels "No","Yes": 2 2 2 2 2 2 2 2 2 2 ...
 $ C.MANIPULATOR: Factor w/ 2 levels "0","1": 2 2 2 2 2 2 2 2 2 2 ...


In [21]:
#Calculating M-score using Beneish model's formula
mscore <- -4.84 + (0.92*ML$DSRI) + (0.528*ML$GMI) + (0.404*ML$AQI) + (0.892*ML$SGI) + 
            (0.115*ML$DEPI) - (0.172*ML$SGAI) + (4.679*ML$ACCR) - (0.327*ML$LEVI)

#Predicting manipulator by setting threshold as -2.22
manip <- ifelse(mscore>-2.22,"Yes","No")
manip <- as.factor(manip)

#Developing a confusion matrix we see that Sensitivity is 100
ML$Manipulater <- relevel(ML$Manipulater, ref = "Yes")
manip <- relevel(manip, ref = "Yes")
a <- confusionMatrix(manip, ML$Manipulater)

#Thus in the existing dataset False negatives (Manipulators predicted as non-manipulators) are zero.

a$table

          Reference
Prediction Yes  No
       Yes  39 371
       No    0 829

In [18]:
a$byClass

Sensitivity          Specificity       Pos Pred Value 
          1.00000000           0.69083333           0.09512195 
      Neg Pred Value            Precision               Recall 
          1.00000000           0.09512195           1.00000000 
                  F1           Prevalence       Detection Rate 
          0.17371938           0.03147700           0.03147700 
Detection Prevalence    Balanced Accuracy 
          0.33091203           0.84541667

In [22]:
#Splitting the data set into Manipulators and non-manipulators
Yes <- ML[ML$Manipulater=="Yes",]
No <- ML[ML$Manipulater=="No",]

#Using sample to determine values randomly for setting seeds while selecting rows from Non-manipulators dataset
set.seed(123)
a <- sample(10000, 100, replace = F)
m <- matrix(NA, ncol = 9, nrow = 100)

In [23]:
for (i in 1:100) {
  set.seed(a[i])
  row <- sample(nrow(No), 220-39, replace = F)
  s <- rbind(Yes, No[row,])
  s <- s[sample(nrow(s)),]
  s <- s[,-c(1,11)]
  sam <- ovun.sample(Manipulater ~ ., data = s, method = "under", N = (nrow(Yes)*2), seed = 1)$data
  
  full <- glm(Manipulater~., data=sam, family = "binomial")
  null <- glm(Manipulater~1, data=sam, family = "binomial")
  b <- step(null, scope = list(lower=null, upper=full), direction = "both")
  c <- summary(b)
  d <- sort(rownames(c$coefficients))
  
  if(length(d)!=9)
  {
    for(j in 1:(9-length(d)))
    {
      d <- c(d,NA)
    }
  }
  
  m[i,] <- d
}

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”

Start:  AIC=110.13
Manipulater ~ 1



Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”

       Df Deviance    AIC
+ DSRI  1   96.745 100.75
+ ACCR  1   99.702 103.70
+ AQI   1  103.066 107.07
+ SGI   1  103.548 107.55
+ SGAI  1  104.809 108.81
<none>     108.131 110.13
+ LEVI  1  106.921 110.92
+ GMI   1  106.993 110.99
+ DEPI  1  107.907 111.91


Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”

...

       Df Deviance     AIC
+ GMI   1   62.659  76.659
<none>      66.425  78.425
+ LEVI  1   64.963  78.963
- ACCR  1   69.442  79.442
+ SGAI  1   65.753  79.753
- DEPI  1   77.460  87.460
- AQI   1   87.946  97.946
- SGI   1   91.644 101.644
- DSRI  1   96.509 106.509


Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


Step:  AIC=76.66
Manipulater ~ DSRI + SGI + AQI + DEPI + ACCR + GMI



Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”

       Df Deviance     AIC
<none>      62.659  76.659
+ SGAI  1   60.764  76.764
+ LEVI  1   61.562  77.562
- GMI   1   66.425  78.425
- ACCR  1   67.124  79.124
- DEPI  1   70.377  82.377
- SGI   1   83.188  95.188
- AQI   1   85.906  97.906
- DSRI  1   94.583 106.583


In [25]:
table(m)

m
(Intercept)        ACCR         AQI        DEPI        DSRI         GMI 
        100         100         100          31         100          86 
       LEVI        SGAI         SGI 
         21           7         100 